# Generate Neural Network Features for UWNDC2019 Model 

<br><br>
### Uses Alexnet and PyTorch

In [13]:
import numpy as np
import warnings
import pandas as pd
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from collections import namedtuple
from tqdm import trange, tqdm


warnings.simplefilter(action='ignore',category=FutureWarning)

datadir ='./Hackathon/v4-challenge/data/' # set this to the location of your data directory which
                   # should contain train.csv and stim.npy

In [14]:
df_now = pd.read_csv(datadir+'train.csv')
im_now = np.load(datadir+'stim.npy')

test_ims = im_now[50:,...]
train_ims = im_now[50:,...]

#calculate mean for image normalization
im_mean = np.mean(np.mean(im_now,axis=(1,2)),axis=0)
im_std = np.std(np.std(im_now,axis=(1,2)),axis=0)


In [15]:
class v4_dataset(Dataset):
    
    def __init__(self, df,ims, transform):
        
        self.resp_frame = df
        self.transform = transform
        self.ims = ims
        
    def __len__(self):
        return len(self.resp_frame)
    
    def __getitem__(self,idx):
        img = Image.fromarray(np.uint8(self.ims[idx,:,:,:]*255))
        responses = self.resp_frame.iloc[idx,1:].as_matrix()
        
        if self.transform is not None:
            img = self.transform(img)
            
        sample = {'image': img, 'responses': responses}
        
        return sample

tfm = transforms.Compose([transforms.Resize(224),transforms.ToTensor(), transforms.Normalize(mean=im_mean,std=im_std)])

train_set = v4_dataset(df_now,train_ims,tfm)
trainloader = DataLoader(train_set,batch_size=4)

test_set = v4_dataset(df_now,test_ims,tfm)
testloader = DataLoader(test_set,batch_size=4)

In [ ]:
class AlexNet(torch.nn.Module):
    
    def __init__(self):
        super(AlexNet,self).__init__()
        features = list(models.alexnet(pretrained=True).features)
        self.features = nn.ModuleList(features).eval()
        
    def forward(self,x):
        results =[]
        for ii, cv_layer in enumerate(self.features):
            x = cv_layer(x)
            if ii in {0, 3, 6, 8, 10}: #just conv1-5
                results.append(x)
        
        an_outputs = namedtuple("AlexNetOutputs",['conv1','conv2','conv3','conv4','conv5'])
        return an_outputs(*results)
    
net = AlexNet()
net.eval()

conv_train = dict()
conv_test = dict()

device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
net.to(device)

for i, data in enumerate(trainloader):
    
    inputs = data['image']
    inputs = inputs.to(device)
    outputs = net(inputs)
    
    if i == 0:
        for j, fld in enumerate(outputs._fields):
            #import pdb; pdb.set_trace()
            conv_train[fld] = outputs[j].cpu().detach().numpy()
    else:
        for j, fld in enumerate(outputs._fields):
            conv_train[fld] = np.vstack((conv_train[fld],outputs[j].cpu().detach().numpy()))
   
    print('{}% done with train examples'.format(round((i+1)*100/len(trainloader),2)))


for i, data in enumerate(testloader):
    
    inputs = data['image']
    inputs = inputs.to(device)
    outputs = net(inputs)
    
    if i == 0:
        for j, fld in enumerate(outputs._fields):
            #import pdb; pdb.set_trace()
            conv_test[fld] = outputs[j].cpu().detach().numpy()
    else:
        for j, fld in enumerate(outputs._fields):
            conv_test[fld] = np.vstack((conv_test[fld],outputs[j].cpu().detach().numpy()))
            
    print('{}% done with test examples'.format(round((i+1)*100/len(testloader),2)))


0.7246376811594203% done with train examples
1.4492753623188406% done with train examples
2.1739130434782608% done with train examples
2.898550724637681% done with train examples
3.6231884057971016% done with train examples
4.3478260869565215% done with train examples
5.072463768115942% done with train examples
5.797101449275362% done with train examples
6.521739130434782% done with train examples
7.246376811594203% done with train examples
7.971014492753623% done with train examples
8.695652173913043% done with train examples
9.420289855072463% done with train examples
10.144927536231885% done with train examples
10.869565217391305% done with train examples
11.594202898550725% done with train examples
12.318840579710145% done with train examples
13.043478260869565% done with train examples
13.768115942028986% done with train examples
14.492753623188406% done with train examples
15.217391304347826% done with train examples
15.942028985507246% done with train examples
16.666666666666668

35.507246376811594% done with test examples
36.231884057971016% done with test examples
36.95652173913044% done with test examples
37.68115942028985% done with test examples
38.405797101449274% done with test examples
39.130434782608695% done with test examples
39.85507246376812% done with test examples
40.57971014492754% done with test examples
41.30434782608695% done with test examples
42.028985507246375% done with test examples
42.7536231884058% done with test examples
43.47826086956522% done with test examples
44.20289855072464% done with test examples
44.927536231884055% done with test examples
45.65217391304348% done with test examples
46.3768115942029% done with test examples
47.10144927536232% done with test examples
47.82608695652174% done with test examples
48.55072463768116% done with test examples
49.27536231884058% done with test examples
50.0% done with test examples
50.72463768115942% done with test examples
51.44927536231884% done with test examples
52.17391304347826% d